# Dinamic stochastic models in Python

## Forecasting non-observables

1. Load libraries

In [1]:
import sys
sys.path.append('Libraries')
import DSGE_LinearMethods as DLM
import DSGE_PreProcessing as DPP
import HandleTimeSeries as HTS
import sympy as sp
sp.init_session(use_latex=True)

IPython console for SymPy 1.12 (Python 3.12.4-64-bit) (ground types: python)

These commands were executed:
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.12/



* Check Libraries below are actually needed

In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime

1. Procesing and solving the model

In [3]:
Mod = DPP.process_file3('MPTBolsillo.txt')
Mod = DLM.SolveModel(Mod, "Cal_MPTBolsillo.txt", True)

-> Stability conditions met

Steady state
            User supplied  Numerical
DY_eq                5.30       5.30
DYs_eq               2.10       2.10
Ds                   0.00       0.00
y                    0.00       0.00
ys                   0.00       0.00
Dpae                 2.00       2.00
Dp                   2.00       2.00
Dp(-2)               2.00       2.00
Dp(-1)               2.00       2.00
D4p                  2.00       2.00
Dpsae                2.00       2.00
Dpsae(-2)            2.00       2.00
Dpsae(-1)            2.00       2.00
ED4p                 2.00       2.00
Dpm                  2.00       2.00
Dpm(-2)              2.00       2.00
Dpm(-1)              2.00       2.00
Meta                 2.00       2.00
i                    3.75       3.75
ieq                  3.75       3.75
zmn                  1.75       1.75
zme                  0.00       0.00
Ds(-2)               0.00       0.00
Ds(-1)               0.00       0.00
ED4s                 0.00       0

3. Load data

In [4]:
Describ, DBobs = HTS.ReadDB_excel('DataTarea.xlsx', sheet='VarObs')

In [5]:
var = DLM.sympy_to_list(Mod['Desc.']['Declared Endogenous'])
varexo = DLM.sympy_to_list(Mod['Desc.']['Shocks'])

In [6]:
DF_s, DF_u, minuslogL = DLM.KFts(DBobs, Mod, {'diffuse':False})

In [7]:
SS = Mod['Solution']['Y0'].reshape(-1,1)

In [8]:
Data = DF_s

# Forecast

1. Unconditional

In [9]:
FH = 8
FDB = DLM.DSGEforecast0(Mod,Data,FH,'2025Q1')

2. Not anticipated conditional forecast

In [ ]:
scenarios = [
    "Incertidumbre política frente a elecciones",
    "Choque de oferta en el sector primario por oleajes intensos",
    "Incertidumbre mundial debido a políticas fuertemente proteccionistas en EUA",
    "Choque de demanda interna debido a vacancia presidencial",
    "Fenómeno del Niño",
    "Choque de demanda externa positivo por mayor crecimiento de China",
    "Expansión de la demanda externa ante una Fed más dovish",
    "Expansión del gasto e inversión pública durante el nuevo gobierno",
    "Alta volatilidad en los mercados financieros globales",
    "Choque de demanda externa negativo por menor crecimiento de China",
    "Contracción de la demanda externa ante una Fed más hawkish",
    "Contracción del gasto e inversión pública",
    "Mayor apetito por riesgo en los mercados financieros globales"
]

results = pd.DataFrame(columns=[
    "Escenario", 
    "Inflación Libre 2025", 
    "Inflación Riesgos 2025", 
    "Impacto 2025", 
    "Inflación Libre 2026", 
    "Inflación Riesgos 2026", 
    "Impacto 2026"
])

for i in range(1, 14):
    UCond = HTS.ReadForeCond(f'Riesgo/DataTarea_Exogenas_Riesgo{i}.xlsx', '2025Q1', FH)
    UCond['Uvar']['var'].i = FDB.i[-8:].values

    UFDB = DLM.DSGEforecast0(Mod, Data, FH, '2025Q1', UCond)

    inflacion_libre_2025 = FDB.D4p[-5:-4][0]
    inflacion_libre_2026 = FDB.D4p[-1:][0]
    inflacion_riesgos_2025 = UFDB.D4p[-5:-4][0]
    inflacion_riesgos_2026 = UFDB.D4p[-1:][0]

    impacto_2025 = round((inflacion_libre_2025 - inflacion_riesgos_2025) * 0.10, 2)
    impacto_2026 = round((inflacion_libre_2026 - inflacion_riesgos_2026) * 0.10, 2)

    new_row = pd.DataFrame({
        "Escenario": [scenarios[i-1]],
        "Inflación Libre 2025": [inflacion_libre_2025],
        "Inflación Riesgos 2025": [inflacion_riesgos_2025],
        "Impacto 2025": [impacto_2025],
        "Inflación Libre 2026": [inflacion_libre_2026],
        "Inflación Riesgos 2026": [inflacion_riesgos_2026],
        "Impacto 2026": [impacto_2026]
    })
    results = pd.concat([results, new_row], ignore_index=True)

results.to_excel("Impactos_Riesgo.xlsx", index=False)

C:\Users\2663\AppData\Local\Temp\ipykernel_23084\397931806.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  inflacion_libre_2025 = FDB.D4p[-5:-4][0]
C:\Users\2663\AppData\Local\Temp\ipykernel_23084\397931806.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  inflacion_libre_2026 = FDB.D4p[-1:][0]
C:\Users\2663\AppData\Local\Temp\ipykernel_23084\397931806.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  inflacion_riesgos_20